<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/07%20-%20Topic%20modelling/topic_modelling_article_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic modelling con Scikit-learn
Latent Dirichlet allocation (LDA) è un modello statistico che ci permette di associare ogni documento del nostro corpus di testo a degli argomenti (topic) quantificando quanto il documento è inerente a tale argomento. Ad esempio, prendiamo la frase: "Il calciatore e la velina si sono spostati a Parigi", l'analisi LDA potrebbe stabilire che la frase riguarda al 80% gossip e al 20% sport. I topic del LDA non contengono il nome specifico dell'argomento (come in questo caso sport o gossip) ma le parole chiave del topic, quindi sta a noi utilizzando queste risalire al topic. 

In questo notebook cerchermo di identificare gli argomenti di quasi 10mila articoli tratti dal New York Times utilizzando LDA con scikit-learn.


## Installare le API di Kaggle
Il dataset contenente le notizie estratte dal New York Times è presente su Kaggle a [questo indirizzo](https://www.kaggle.com/nzalake52/new-york-times-articles), per scaricarlo devi creare un'account gratuito. per scaricare l'ultima versione aggiornata devi registrarti su Kaggle. Se utilizzi Google Colab può essere utile scaricare il dataset usando le API di Kaggle. Cominciamo installando il modulo kaggle usando pip (dovrebbe essere già installato, ma facciamolo comunque per sicurezza).



In [3]:
!pip install kaggle

Per utilizzare le API di Kaggle dobbiamo creare un API Token, puoi farlo così:
1. Recati su Kaggle ed effettua il login.
2. Clicca sulla tua immagine di profilo in alto a destra.
3. Seleziona "My Account" dal menu a tendina.
4. Recati nella sezione API
5. Clicca su "Create new API Token"

Un file kaggle.json verrà scaricato sul tuo pc, aprilo ed incolla i valori per username nel codice qui sotto, ci servirà per generare il file .kaggle necessario per l'utilizzo delle API.

In [0]:
from os import listdir

user = "INCOLLA QUI IL TUO USERNAME"
key = "INCOLLA QUI LA TUA API TOKEN"

if '.kaggle' not in listdir('/root'):
    !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (user, key))
!chmod 600 /root/.kaggle/kaggle.json

## Scarichiamo il dataset

Scarichiamo il dataset usando le API di Kaggle.

In [5]:
!kaggle datasets download nzalake52/new-york-times-articles

 30% 5.00M/16.9M [00:00<00:00, 21.2MB/s]
100% 16.9M/16.9M [00:00<00:00, 56.3MB/s]


Estraimo il file zip usando unzip.

In [6]:
!unzip new-york-times-articles.zip

Archive:  new-york-times-articles.zip
  inflating: nytimes_news_articles.txt  


Ora possiamo aprire il file txt usando python.

In [116]:
news_file = open("nytimes_news_articles.txt")
news = news_file.read()

print(news[:1000])

URL: http://www.nytimes.com/2016/06/30/sports/baseball/washington-nationals-max-scherzer-baffles-mets-completing-a-sweep.html

WASHINGTON — Stellar pitching kept the Mets afloat in the first half of last season despite their offensive woes. But they cannot produce an encore of their pennant-winning season if their lineup keeps floundering while their pitching is nicked, bruised and stretched thin.
“We were going to ride our pitching,” Manager Terry Collins said before Wednesday’s game. “But we’re not riding it right now. We’ve got as many problems with our pitching as we do anything.”
Wednesday’s 4-2 loss to the Washington Nationals was cruel for the already-limping Mets. Pitching in Steven Matz’s place, the spot starter Logan Verrett allowed two runs over five innings. But even that was too large a deficit for the Mets’ lineup to overcome against Max Scherzer, the Nationals’ starter.
“We’re not even giving ourselves chances,” Collins said, adding later, “We just can’t give our pitcher

## Preprocessiamo i dati
Ogni articolo è introdotto dall'URL, usiamo questo pattern per dividere gli articoli.

In [117]:
import re

news_split = re.split("URL: http://www.nytimes.com/\S+", news)
del news_split[0] # il primo elemento è vuoto, rimuoviamolo 
print(len(news_split))

8888


Abbiamo 8888 articoli, stampiamo il primo.

In [118]:
news_split[0]

'\n\nWASHINGTON — Stellar pitching kept the Mets afloat in the first half of last season despite their offensive woes. But they cannot produce an encore of their pennant-winning season if their lineup keeps floundering while their pitching is nicked, bruised and stretched thin.\n“We were going to ride our pitching,” Manager Terry Collins said before Wednesday’s game. “But we’re not riding it right now. We’ve got as many problems with our pitching as we do anything.”\nWednesday’s 4-2 loss to the Washington Nationals was cruel for the already-limping Mets. Pitching in Steven Matz’s place, the spot starter Logan Verrett allowed two runs over five innings. But even that was too large a deficit for the Mets’ lineup to overcome against Max Scherzer, the Nationals’ starter.\n“We’re not even giving ourselves chances,” Collins said, adding later, “We just can’t give our pitchers any room to work.”\nThe Mets did not score until the ninth inning, when a last-gasp two-run homer by James Loney off 

Rimuoviamo i caratteri di a capo da ogni articolo.

In [42]:
import string

#news_split = [news.replace("\n","").lower().translate((str.maketrans('', '', string.punctuation))) for news in news_split]
news_split = [news.replace("\n","") for news in news_split]
news_split[0]

'WASHINGTON — Stellar pitching kept the Mets afloat in the first half of last season despite their offensive woes. But they cannot produce an encore of their pennant-winning season if their lineup keeps floundering while their pitching is nicked, bruised and stretched thin.“We were going to ride our pitching,” Manager Terry Collins said before Wednesday’s game. “But we’re not riding it right now. We’ve got as many problems with our pitching as we do anything.”Wednesday’s 4-2 loss to the Washington Nationals was cruel for the already-limping Mets. Pitching in Steven Matz’s place, the spot starter Logan Verrett allowed two runs over five innings. But even that was too large a deficit for the Mets’ lineup to overcome against Max Scherzer, the Nationals’ starter.“We’re not even giving ourselves chances,” Collins said, adding later, “We just can’t give our pitchers any room to work.”The Mets did not score until the ninth inning, when a last-gasp two-run homer by James Loney off Nationals re

## LDA con Bag of Words

Ora dobbiamo codificare gli articoli in numeri, facciamolo usando il modello  bag of words. Due note sui parametri: 
 - Il parametro *lowercase* ci permette di convertire tutto in minuscolo, questo parametro è impostato a True di default, quindi avremmo ance potuto ometterlo.
 - Il parametro *stop_words* ci permette di rimuovere le stop words, come valore dobbiamo passargli la lingua.

In [43]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer(max_features=5000, stop_words="english", lowercase=True)
X = bow.fit_transform(news_split)
X.shape

(8888, 5000)

Adesso creiamo il modello LDA usando la classe *LatentDirichletAllocation* di sklearn, il mumero di topic va definito a priori all'interno del parametro n_components, impostiamolo a 25.

In [46]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

n_topics = 25

lda = LDA(n_components=n_topics, max_iter=10, verbose=True)
lda.fit_transform(X)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


array([[1.25000000e-04, 1.25000000e-04, 1.25000000e-04, ...,
        1.25000000e-04, 1.25000000e-04, 1.25000000e-04],
       [1.61943320e-04, 1.61943320e-04, 1.61943320e-04, ...,
        1.61943320e-04, 7.32124844e-01, 1.20517011e-01],
       [1.46937068e-01, 1.23456790e-04, 3.84998573e-02, ...,
        1.23456790e-04, 6.80162086e-02, 5.84040454e-01],
       ...,
       [1.35160148e-01, 8.84955752e-05, 8.84955752e-05, ...,
        1.95213549e-01, 2.59539277e-02, 8.84955752e-05],
       [5.40540541e-04, 5.40540541e-04, 5.40540541e-04, ...,
        5.40540541e-04, 5.40540541e-04, 3.30610734e-01],
       [3.07517438e-01, 9.85221675e-05, 9.85221675e-05, ...,
        9.85221675e-05, 9.85221675e-05, 1.20425170e-01]])

Il modello è pronto ! Cerchiamo di farci un'idea di cosa i nostri topic rappresentano visualizzando le 10 parole più comuni per ognuno.

In [48]:
n_words = 10

for index, topic in enumerate(lda.components_):
  print("\nTOPIC %d - %d parole più popolari" % (index+1, n_words))
  print([bow.get_feature_names()[i] for i in topic.argsort()[-n_words:]])


TOPIC 1 - 10 parole più popolari
['area', 'mr', 'people', 'water', 'park', 'building', '000', 'new', 'city', 'said']

TOPIC 2 - 10 parole più popolari
['coach', 'play', 'points', 'league', 'year', 'games', 'said', 'players', 'game', 'team']

TOPIC 3 - 10 parole più popolari
['racing', 'world', 'year', 'horse', 'derby', 'muhammad', 'briefing', 'said', 'race', 'ali']

TOPIC 4 - 10 parole più popolari
['innings', 'inning', 'runs', 'home', 'second', 'hit', 'season', 'run', 'game', 'mets']

TOPIC 5 - 10 parole più popolari
['did', 've', 'team', 'think', 'time', 'like', 'going', 'game', 'just', 'said']

TOPIC 6 - 10 parole più popolari
['son', 'couple', 'bride', 'san', 'received', 'father', 'university', 'graduated', 'york', 'new']

TOPIC 7 - 10 parole più popolari
['way', 'time', 'work', 'just', 'art', 'new', 'people', 'said', 'mr', 'like']

TOPIC 8 - 10 parole più popolari
['team', 'right', 'baseball', 'miller', 'game', 'season', 'girardi', 'rodriguez', 'said', 'yankees']

TOPIC 9 - 10 pa

Alcuni topic sono molto chiari:
 - il Topic 10 ha a che fare con l'editoria e le notizie.
 - i Topic 2, 3, 4 e 8 hanno a che fare con lo sport. 
 - il Topic 12 ha a che fare con la finanza
 - il Topic 20 ha a che fare con la tecnologia
 - il Topic 25 ha a che fare con la cronaca nera.

Definiamo una funzione per classificare nuovi articoli all'interno di questi topic.

In [0]:
import numpy as np

def classify(text, return_proba=False):
  x = bow.transform([text])
  y_proba = lda.transform(x)[0]
  y = y_proba.argmax()
  
  if(return_proba):
    topics_ordered = np.argsort(-y_proba)
    y_proba = -np.sort(-y_proba)
    y_dict = dict(["TOPIC: "+str(topic+1), proba] for topic, proba in zip(topics_ordered, y_proba))
    return y, y_dict
  
  return y

Testiamo con un semplice articolo che riguarda la tecnologia.

In [108]:
# Notiza dell'ultima ora ! Samsung ha appena svelato il suo nuovo smartphone, il Galaxy S10
my_news = "Breaking News! Samsung just unveil its new smartphone, the new Galaxy S10"

y, y_proba = classify(my_news, return_proba=True)

print("Topic di appartenenza: %d" % (y+1))
print("\n")
for topic in y_proba:
  print("%s = %.4f" % (topic, y_proba[topic]))

Topic di appartenenza: 20


TOPIC: 20 = 0.6238
TOPIC: 10 = 0.2448
TOPIC: 5 = 0.0057
TOPIC: 18 = 0.0057
TOPIC: 7 = 0.0057
TOPIC: 8 = 0.0057
TOPIC: 9 = 0.0057
TOPIC: 3 = 0.0057
TOPIC: 24 = 0.0057
TOPIC: 19 = 0.0057
TOPIC: 23 = 0.0057
TOPIC: 11 = 0.0057
TOPIC: 6 = 0.0057
TOPIC: 16 = 0.0057
TOPIC: 4 = 0.0057
TOPIC: 1 = 0.0057
TOPIC: 14 = 0.0057
TOPIC: 25 = 0.0057
TOPIC: 17 = 0.0057
TOPIC: 13 = 0.0057
TOPIC: 2 = 0.0057
TOPIC: 12 = 0.0057
TOPIC: 15 = 0.0057
TOPIC: 22 = 0.0057
TOPIC: 21 = 0.0057


Ottimo ! Facciamo un'altro testo con un'articolo che riguarda tecnologia e cronaca nera.

In [114]:
# Un'auto a guida autonoma di Apple è andata a sbattere durante un test, due uomini feriti
my_news = "An Apple self driving car crashed during a test, two men injured"
y, y_proba = classify(my_news, return_proba=True)

print("Topic di appartenenza: %d" % (y+1))
print("\n")
for topic in y_proba:
  print("%s = %.4f" % (topic, y_proba[topic]))

Topic di appartenenza: 25


TOPIC: 25 = 0.4052
TOPIC: 21 = 0.2527
TOPIC: 20 = 0.2322
TOPIC: 7 = 0.0050
TOPIC: 14 = 0.0050
TOPIC: 23 = 0.0050
TOPIC: 8 = 0.0050
TOPIC: 2 = 0.0050
TOPIC: 5 = 0.0050
TOPIC: 3 = 0.0050
TOPIC: 22 = 0.0050
TOPIC: 4 = 0.0050
TOPIC: 15 = 0.0050
TOPIC: 17 = 0.0050
TOPIC: 11 = 0.0050
TOPIC: 10 = 0.0050
TOPIC: 16 = 0.0050
TOPIC: 1 = 0.0050
TOPIC: 19 = 0.0050
TOPIC: 18 = 0.0050
TOPIC: 13 = 0.0050
TOPIC: 12 = 0.0050
TOPIC: 24 = 0.0050
TOPIC: 6 = 0.0050
TOPIC: 9 = 0.0050


Come vedi in questo caso la notizia è stata classificata come appartenente al 40% al Topic 25 (cronaca nera) al 25% al Topic 21 (Salute/Legge) e al 23% al Topic 20 (Tecnologia). Un'ottimo risultato :)

## LDA con TF-IDF
Proviamo a ricreare il nostro modello, questa volta usando come codifica il TF-IDF

In [119]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000, stop_words="english")
X = tfidf.fit_transform(news_split)
X.shape

(8888, 5000)

In [120]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

n_topics = 25

lda = LDA(n_components=n_topics, max_iter=10, verbose=True)
lda.fit_transform(X)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


array([[0.00378589, 0.00378589, 0.00378589, ..., 0.00378589, 0.00378589,
        0.00378589],
       [0.00353439, 0.00353439, 0.00353439, ..., 0.00353439, 0.00353439,
        0.00353439],
       [0.00348255, 0.00348255, 0.00348255, ..., 0.00348255, 0.00348255,
        0.00348255],
       ...,
       [0.00321652, 0.00321652, 0.00321652, ..., 0.00321652, 0.00321652,
        0.00321652],
       [0.00501441, 0.00501441, 0.00501441, ..., 0.00501441, 0.00501441,
        0.00501441],
       [0.00287467, 0.00287467, 0.00287467, ..., 0.00287467, 0.00287467,
        0.00287467]])

In [121]:
n_words = 10

for index, topic in enumerate(lda.components_):
  print("\nTOPIC %d - %d parole più popolari" % (index+1, n_words))
  print([bow.get_feature_names()[i] for i in topic.argsort()[-n_words:]])


TOPIC 1 - 10 parole più popolari
['844', 'income', '212', 'mail', '698', '556', 'columbia', 'mississippi', 'nyquist', 'articles']

TOPIC 2 - 10 parole più popolari
['testing', 'owner', 'cheap', 'diego', 'carry', 'vehicle', 'emerging', 'theory', 'christie', 'voices']

TOPIC 3 - 10 parole più popolari
['pulling', 'twice', 'nation', 'center', 'telephone', 'children', 'searching', 'asia', 'behavior', 'performers']

TOPIC 4 - 10 parole più popolari
['revival', 'spurs', 'robust', 'equivalent', 'shelter', 'lemonade', 'criticized', 'detroit', 'pulling', 'twice']

TOPIC 5 - 10 parole più popolari
['revival', 'spurs', 'robust', 'equivalent', 'shelter', 'lemonade', 'criticized', 'detroit', 'pulling', 'twice']

TOPIC 6 - 10 parole più popolari
['publishing', 'global', 'jordan', 'performing', 'oversight', 'lighting', 'pitsiladis', 'shares', 'island', 'pending']

TOPIC 7 - 10 parole più popolari
['74', 'illegal', 'cnn', 'defeated', 'string', 'wrestling', 'wright', 'philippines', 'built', 'hasn']

T

Lascio a te l'interpretazione dei topic.

## Visualizzare il modello
[pyLDAvis](https://github.com/bmabey/pyLDAvis) è un fantastico modulo python che ci permette di esplorare i topic generati da un modello LDA in maniera visuale, installiamolo usando pip.



In [122]:
!pip install pyldavis

     |████████████████████████████████| 1.6MB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis


Utilizziamo per creare la visualizzazione per un modello sklearn, passando come argomenti il modello stesso, il dataset codificato e l'oggetto che abbiamo usato per la codifica. Per poter visualizzare il grafico dobbiamo proiettare i dati in uno spazio bi-dimensionale, all'interno del parametro mds possiamo definire la tecnica per farlo, utilizziamo il t-distributed stochastic neighbor embedding.

In [123]:
import pyLDAvis.sklearn

lda_viz = pyLDAvis.sklearn.prepare(lda, X, tfidf, mds='tsne')
pyLDAvis.display(lda_viz)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
